<a href="https://colab.research.google.com/github/hjuuujh/-/blob/master/2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#데이터 다운위해 캐글 연결
!pip install kaggle
from google.colab import files
files.upload()

In [0]:
ls -1ha kaggle.json

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [0]:
#파일 다운로드
!kaggle datasets download -d praveengovi/coronahack-chest-xraydataset

In [0]:
#현재 파일 확인
!ls

In [0]:
#압축해제
!unzip coronahack-chest-xraydataset.zip

In [0]:
import os
import numpy as np
import torch
from PIL import Image

In [0]:
#파일 경로
train_path='Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/train/'
test_path='Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/test/'

In [0]:
#train 파일 리스트에서 normal 데이터 삭제
train_files=np.array(os.listdir(train_path))
print(train_files[0:10])

idx1=[]
i=0
for i in range(len(train_files)):
  if train_files[i][0:2]=='IM':
    idx1.append(i)
train_files=np.delete(train_files, idx1)
print(train_files[0:10])

idx2=[]
j=0
for j in range(len(train_files)):
  if train_files[j][0:2]=='NO':
    idx2.append(j)
train_files=np.delete(train_files, idx2)
print(train_files[0:10])

In [0]:
#test 파일 리스트에서 normal 데이터 삭제
test_files=np.array(os.listdir(test_path))
print(test_files[0:10])

idx1=[]
i=0
for i in range(len(test_files)):
  if test_files[i][0:2]=='IM':
    idx1.append(i)
test_files=np.delete(test_files, idx1)
print(test_files[0:10])

idx2=[]
j=0
for j in range(len(test_files)):
  if test_files[j][0:2]=='NO':
    idx2.append(j)
test_files=np.delete(test_files, idx2)
print(test_files[0:10])

In [0]:
#이미지 파일 열기, 각 이미지 (128,128,2) shape으로 저장
data=[]

for f in test_files:
  img = Image.open(test_path + f).convert("LA")
  resize_img = img.resize((128, 128))
  re_img_arr=np.asarray(resize_img)/255.0
  data.append(re_img_arr)

print(np.array(data).shape)

In [0]:
for f in train_files:
  img = Image.open(train_path + f).convert("LA")
  resize_img = img.resize((128, 128))
  re_img_arr=np.asarray(resize_img)/255.0
  data.append(re_img_arr)

data=np.array(data)
print(data.shape)

In [0]:
data=torch.tensor(data)
X_train, X_test = train_test_split(data, 
                                  test_size=0.4, 
                                  shuffle=True, 
                                  random_state=1004)

print(X_train.shape)
print(X_test.shape)

X_train = X_train.reshape(X_train.shape[0], -1) 
X_test = X_test.reshape(X_test.shape[0], -1)

ones_train = torch.ones(X_train.shape[0], 1, device=X_train.device,dtype=torch.double)
X_train = torch.cat([X_train, ones_train], dim=1)
ones_test = torch.ones(X_test.shape[0], 1, device=X_test.device,dtype=torch.double)
X_test = torch.cat([X_test, ones_test], dim=1)

validation_ratio = 0.02
num_training = int( X_train.shape[0] * (1.0 - validation_ratio) )
num_validation = X_train.shape[0] - num_training

data_dict = {}
data_dict['X_val'] = X_train[num_training:num_training + num_validation]
#data_dict['y_val'] = y_train[num_training:num_training + num_validation]
data_dict['X_train'] = X_train[0:num_training]
#data_dict['y_train'] = y_train[0:num_training]

data_dict['X_test'] = X_test
#data_dict['y_test'] = y_test

print('-'*40)
print(X_train.shape)
print(X_test.shape)